In [2]:
from keras.models import load_model
from LJT_database.feature_prep import feature_preperation
from LJT_database.merge_dataset import retrieve_data_predictors
from LJT_helper_functions.dataset_prep import prepare_dataset_prediction

from LJT_database.firestore_codes import  add_update,retrieve_updates

from LJT_database.merge_dataset import retrieve_data_predictors

from LJT_database.feature_prep import feature_preperation
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [24]:
model = load_model("final_models/final_model_1_min_LSTM_retrained_25_6.h5")

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 75)                39000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 75)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 152       
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 39,152
Trainable params: 39,152
Non-trainable params: 0
_________________________________________________________________


In [26]:
settings_analysis = {
    "algoirthm":"LSTM",
    "experiment_date":"6_25",#"11_6",
    "neurons":75,
    "dropout":0.18498640636327568,
    "loss_func": "binary_crossentropy",
    "activation_function":"sigmoid",
    "number_layers":1,
    "batch_size":2120,
    "epochs":550,
    "bidrectional":False,
    "window_size":97,
    "time_ahead_prediction":1,
    "optimizer":'adam',
    "learning_rate": 0.002205456488247791,
}

In [27]:
general_settings=retrieve_updates(dataset=f"ETHBTC_LSTM_experiments",
                                        document="experiment_general_settings"
                                        )
#general_settings['columns'] = "*"
##general_settings['last_timestamp'] = 1591747200000
#general_settings['experiment_date'] = "6_11_full_dataset_manual"

In [6]:
dataset_prepared = feature_preperation(retrieve_data_predictors(general_settings)) #retrieve and clea

This requests costed: 332.79 mb


In [12]:
from deep_learning_models.feature_selection import create_reduced_features
df = create_reduced_features(dataset_prepared,general_settings)

Blob ethbtc/6_25_scaler.p downloaded to temp_store/ethbtc/6_25_scaler.p.
Blob ethbtc/6_25_pca.p downloaded to temp_store/ethbtc/6_25_pca.p.
Blob ethbtc/6_25_columns_scaler.p downloaded to temp_store/ethbtc/6_25_columns_scaler.p.
Blob ethbtc/6_25_columns_in_pca.p downloaded to temp_store/ethbtc/6_25_columns_in_pca.p.


In [13]:
#df = pd.read_csv("dataset_full_save.csv")

In [14]:
#df = df[[col for col in df.columns if 'Unname' not in col]]

In [16]:
df.columns

Index(['column_0', 'column_1', 'column_2', 'column_3', 'column_4', 'column_5',
       'column_6', 'column_7', 'column_8', 'column_9', 'column_10',
       'column_11', 'column_12', 'column_13', 'column_14', 'column_15',
       'column_16', 'column_17', 'column_18', 'column_19', 'column_20',
       'column_21', 'column_22', 'column_23', 'column_24', 'column_25',
       'column_26', 'column_27', 'column_28', 'column_29', 'column_30',
       'column_31', 'column_32', 'column_33', 'column_34', 'column_35',
       'column_36', 'column_37', 'column_38', 'column_39', 'column_40',
       'column_41', 'column_42', 'column_43', 'column_44', 'column_45',
       'column_46', 'column_47', 'column_48', 'column_49', 'column_50',
       'column_51', 'column_52', 'last_start_time',
       'ETHBTC__ticker_info__close_price'],
      dtype='object')

In [22]:
from deep_learning_models.training_models import build_model
model = build_model(settings_analysis,general_settings,df,save_model=True,normalized=True)
model.save("final_model_1_min_LSTM_long_1_1_new.h5")

---------------------------------NEW EXPERIMENT -----------------------------------------------
current time: 21:53
--data--
length training data =  143548
percentage True train = 0.47679521832418426
length test data =  35888
percentage True test = 0.468067320552831
highest_val_ac 0.5498356223106384
{'sensitivity': 0.3495654244552923, 'specificity': 0.7261917234154007, 'precision': 0.5290566717722317, 'accuracy': 0.5499052608114133, 'f_1': 0.420977166003513}
Running time  0:56:17.397485


In [28]:
from LJT_helper_functions.dataset_prep import prepare_dataset_prediction

X_train, X_test, y_train, y_test,dataset_prept = prepare_dataset_prediction(df, {
            "coinpair": general_settings["coin"],
            "window_size": int(settings_analysis["window_size"]),
            "training_size": general_settings["training_size"],
            "bins": general_settings["bins"],
            "time_steps_ahead_prediction": int(settings_analysis["time_ahead_prediction"]),
            "include_transaction_cost": general_settings["include_transaction_cost"]
        }, normalized=True,return_dataset=True)

In [29]:
import numpy as np
from sklearn.metrics import confusion_matrix

def retrieve_scores(model, X_test, y_test):
        x = model.predict(X_test)
        y_pred = np.argmax(x, axis=1)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        sensitivity = tp / (tp + fn)  # recall
        specificity = tn / (fp + tn)
        precision = tp / (tp + fp)
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        f_1 = 2 * ((precision * sensitivity) / (precision + sensitivity))
        return sensitivity, specificity, precision, accuracy, f_1,y_pred
sensitivity, specificity, precision, accuracy, f_1,predictions = retrieve_scores(model, X_test,y_test)    
print(sensitivity, specificity, precision, accuracy, f_1,predictions)

0.3495654244552923 0.7261917234154007 0.5290566717722317 0.5499052608114133 0.420977166003513 [0 0 1 ... 0 0 1]
